In [2]:
import torch
from torch import nn
from torchvision.models.segmentation import fcn_resnet50, FCN_ResNet50_Weights
import cv2
import numpy as np
from albumentations.pytorch.transforms import ToTensorV2
import albumentations as A
from models.unet import UNet
from models.fcn import FCN_DANN, FCN
from shapely.geometry import Polygon

In [2]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()

        weights = FCN_ResNet50_Weights.DEFAULT
        self.model = fcn_resnet50(num_classes=21) # 
        self.conv = nn.Conv2d(21, 3, 1)

    def forward(self, x):
        x = self.model(x)['out']
        x = self.conv(x)
        x = torch.sigmoid(x)

        return x

In [3]:
# model = Model().cuda()
model = FCN(classes=2).cuda()
model.load_state_dict(torch.load('auto_label.pt'))

<All keys matched successfully>

In [4]:
transform_valid = A.Compose(
    [
        A.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
        ToTensorV2(),
    ]
)

In [8]:
# cap 이 정상적으로 open이 되었는지 확인하기 위해서 cap.isOpen() 으로 확인가능
# cap = cv2.VideoCapture('test_video.avi')
cap = cv2.VideoCapture(0)

# 3은 width, 4는 height
cap.set(3,640)
cap.set(4,480)

while(True):
    # ret : frame capture결과(boolean)
    # frame : Capture한 frame
    ret, frame = cap.read()

    if (ret):
        with torch.no_grad():
            frame_copy = frame.copy()
            frame = frame[..., ::-1]
            frame = transform_valid(image=frame)['image']
            # frame = cv2.resize(frame, (640, 360))
            # frame = cv2.normalize(frame, None, 0, 1, cv2. NORM_MINMAX)
            # frame = np.expand_dims(np.transpose(frame, (2, 0, 1)), 0).astype(np.float32)
            frame = torch.unsqueeze(frame, 0)
            frame = torch.tensor(frame).cuda()
            pred = model(frame)
            
        pred = pred.detach().cpu().numpy()
        pred = np.transpose((np.squeeze(pred) > .5) * 255, (1, 2, 0))
        pred = pred.astype(np.uint8)
        pred = pred[..., :-1] # hand: ch1, head: ch2
        # print(pred.dtype, pred.shape, frame_copy.dtype, frame_copy.shape)

        # bbox for hand
        # 1. contour 찾기
        # 2. 가장 큰 거 2 개만 골라 내기
        # 3. bbox 좌표 구하기
        # 4. window 에 bbox 씌우기

        contours, h = cv2.findContours(pred, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_NONE)
        polys = [Polygon(con) for con in contours]
        polys = sorted(polys, key=lambda x:x.area)
        polys = polys[:2]
        for poly in polys:
            x = polys[0][:, 0]
            y = polys[0][:, 0]
            print(x.min(), x.max(), y.min(), y.max())
        cv2.rectangle(frame_copy, (x.min(), y.min()), (x.max(), y.max()), (255, 0, 0), 3)


        cv2.imshow('pred', np.clip(np.sum(pred, axis=-1), 0, 255).astype(np.uint8))
        cv2.imshow('frame', frame_copy)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

C:\Users\HEBB\AppData\Local\Temp\ipykernel_40868\1678523784.py:23: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  frame = torch.tensor(frame).cuda()


In [ ]:
gen = iter([1, 2, 3])
gen

In [ ]:
gen.__next__()

2